In [1]:
%matplotlib

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

cv.destroyAllWindows()

Using matplotlib backend: Qt5Agg


In [2]:
# Leemos los clasificadores pre-entrenados
face_cascade = cv.CascadeClassifier('/home/fran6ko/anaconda3/envs/datos/share/opencv4/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('/home/fran6ko/anaconda3/envs/datos/share/opencv4/haarcascades/haarcascade_eye.xml')
smile_cascade = cv.CascadeClassifier('/home/fran6ko/anaconda3/envs/datos/share/opencv4/haarcascades/haarcascade_smile.xml')

## Static Solution

In [3]:
camera = cv.VideoCapture(0)

ret,frame = camera.read()
cv.imshow('Foto',frame)

camera.release()

In [4]:
def face_eye_smile_detection(img):

    # Cargamos la imagen a analizar
    originalImage = img.copy()

    # Pasamos la imagen a escala de grises
    grayImage = cv.cvtColor(originalImage, cv.COLOR_BGR2GRAY)
    grayImage = cv.equalizeHist(grayImage)

    # Llamamos al clasificador de Haar (AdaBoost)
    faces = face_cascade.detectMultiScale(grayImage, 1.1, 5, minSize=(200, 200))

    # Extra data
    eyes_res = None
    
    # Recorro las caras encontradas
    for (x,y,w,h) in faces:
        # Le dibujamos un rectángulo amarillo
        cv.rectangle(originalImage,(x,y),(x+w,y+h),(255,255,0),4)
        # Definimos las ROIs en la imagen gris y color
        roi_gray = grayImage[y:y+h, x:x+w] 
        roi_color = originalImage[y:y+h, x:x+w]
        # Para cada rostro hallado le buscamos los ojos
        e = int(w/5)
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 7, minSize=(e,e))
        # En los ojos hallados les dibujamos rectángulos
        for (ex,ey,ew,eh) in eyes:
            cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        # Para cada rostro hallado le buscamos las sonrisas
        s = int(w/6)
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.3, 17, minSize=(2*s,s))
        # En las sonrisas halladas les dibujamos rectángulos
        for (ex,ey,ew,eh) in smiles:
            #cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2)
            triangle = np.array([[[ex, ey], [ex+ew, ey], [ex+int(ew/2), ey+eh]]], np.int32)
            cv.polylines(roi_color, [triangle], True, (0,0,255), thickness=3)
        # Devolver ojos si se encontraron exactamente 2
        if len(eyes) == 2:
            eyes_res = eyes + [x, y, 0, 0]
    
    return originalImage, eyes_res
            
res,_ = face_eye_smile_detection(frame)
    
cv.imshow('Deteccion de caras con filtros de Haar en cascada',res) 
cv.waitKey(0)
cv.destroyAllWindows()

## Dynamic Solution

In [5]:
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )

camera = cv.VideoCapture(0)

while(1):
    ret, frame = camera.read()
    
    if ret == True:
        res,_ = face_eye_smile_detection(frame)
        cv.imshow('Live Detection', res) 
        
        # If the ESC kay is pressed, stop the execution
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
        
cv.destroyAllWindows()
camera.release()

## Glasses

In [6]:
qty = 3
lentes = [cv.imread("lentes" + str(i) + ".png", cv.IMREAD_UNCHANGED) for i in range(qty)]

In [7]:
# Funcion para mezclar 2 imagenes (https://www.programcreek.com/python/example/89436/cv2.addWeighted - Examlple 4)

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:, :, :3]  # Grab the BRG planes
    overlay_mask = overlay_t_img[:, :, 3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv.cvtColor(overlay_mask, cv.COLOR_GRAY2BGR)
    background_mask = cv.cvtColor(background_mask, cv.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0)) 

In [8]:
counter = 0
previous_eyes = None

def put_glasses(img, glasses):
    
    global previous_eyes, counter

    res,eyes = face_eye_smile_detection(img)
    
    if eyes is None:
        if previous_eyes is None:
            return img
        else:
            if counter > 5:
                return img
            else:
                counter = counter + 1
                eyes = previous_eyes
    else:
        counter = 0
        previous_eyes = eyes
    
    dist_res = np.linalg.norm(eyes[0,:2] - eyes[1,:2])

    lentes = glasses.copy()
    face = img.copy()

    dist = lentes.shape[1]/2
    scale_factor = dist_res/dist * 1.3
    new_size = (int(lentes.shape[1]*scale_factor), int(lentes.shape[0]*scale_factor))
    lentes = cv.resize(lentes, new_size)

    left_eye = eyes[0] if eyes[0,0] < eyes[1,0] else eyes[1]

    x_offset = left_eye[0] + int(left_eye[2]/2) - int(new_size[0]/3.2)
    y_offset = int((eyes[0,1] +  eyes[1,1])/2) + int(left_eye[3]/2) - int(new_size[1]/2)

    roi = face[y_offset:y_offset+new_size[1], x_offset:x_offset+new_size[0]]
    face[y_offset:y_offset+new_size[1], x_offset:x_offset+new_size[0]] = blend_transparent(roi, lentes)

    return face
    
res = put_glasses(frame, lentes[0])
cv.imshow('Static Glasses',res) 
    
cv.waitKey(0)
cv.destroyAllWindows()

## Dynamic Glasses

In [9]:
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )

camera = cv.VideoCapture(0)

idx = 0

while(1):
    ret, frame = camera.read()
    
    if ret == True:
        res = put_glasses(frame, lentes[idx])
        cv.imshow('Live Glasses', res) 
        
        # If the ESC kay is pressed, stop the execution
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
        
        # Switch glasses with A/D arrows
        if k == 100:
            idx = idx + 1
            if idx == len(lentes):
                idx = 0
        if k == 97:
            idx = idx - 1
            if idx == -1:
                idx = len(lentes) - 1
    else:
        break
        
cv.destroyAllWindows()
camera.release()